Let's first introduce some code based on the img2vec package that will let us get embeddings from a ResNet model for our images.

In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np


DEVICE = torch.device('cpu')
OUTPUT_SIZE = 512

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
extraction_layer = model._modules.get('avgpool')
model.to(DEVICE)
model.eval()

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.2, 0.2, 0.2])
to_tensor = transforms.ToTensor()

def get_vec(arg, model, extraction_layer):
    image = normalize(to_tensor(scaler(arg))).unsqueeze(0).to(DEVICE)
    result = torch.zeros(1, OUTPUT_SIZE, 1, 1)
    def copy_data(m, i, o):
        result.copy_(o.data)
    hooked = extraction_layer.register_forward_hook(copy_data)
    with torch.no_grad():
        model(image)
    hooked.remove()
    return result


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 131MB/s]


Now let's get embeddings using ResNet; this takes about 45 minutes so now would be a good time to eat lunch.

In [2]:
import base64
import pandas as pd
from arrow import now
from glob import glob
from io import BytesIO
from os.path import basename
from PIL import Image

DATA = '/kaggle/input/sign-language-detection-using-images/data'
STOP = 1300
THUMBNAIL_SIZE = (128, 128)


def embed(model, filename: str):
    with Image.open(fp=filename, mode='r') as image:
        return get_vec(arg=image.convert('RGB'), model=model, extraction_layer=extraction_layer).numpy().reshape(OUTPUT_SIZE,)


# https://stackoverflow.com/a/952952
def flatten(arg):
    return [x for xs in arg for x in xs]

def png(filename: str) -> str:
    with Image.open(fp=filename, mode='r') as image:
        buffer = BytesIO()
        # our images are pretty big; let's shrink the hover images to thumbnail size
        image.resize(size=THUMBNAIL_SIZE).convert('RGB').save(buffer, format='png')
        return 'data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode()

def get_picture_from_glob(arg: str, tag: str, stop: int) -> list:
    time_get = now()
    result = [pd.Series(data=[tag, basename(input_file), embed(model=model, filename=input_file), ],
                        index=['tag', 'name', 'value', ])
        for index, input_file in enumerate(glob(pathname=arg)) if index < stop and input_file.endswith('.jpg')]
    print('encoded {} rows of {}  in {}'.format(len(result), tag, now() - time_get))
    return result

time_start = now()
data_dict = {basename(folder) : folder + '/*.*' for folder in glob(DATA + '/*')}
df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key, stop=STOP) for key, value in data_dict.items()]))
print('done in {}'.format(now() - time_start))

encoded 1200 rows of N  in 0:01:19.409469
encoded 1200 rows of 7  in 0:01:18.335808
encoded 1200 rows of R  in 0:01:23.917014
encoded 1200 rows of 2  in 0:01:27.146545
encoded 1200 rows of B  in 0:01:29.910776
encoded 1200 rows of I  in 0:01:32.691886
encoded 1200 rows of F  in 0:01:24.136629
encoded 1200 rows of H  in 0:01:23.389766
encoded 1200 rows of 5  in 0:01:24.469341
encoded 1200 rows of E  in 0:01:20.907008
encoded 1200 rows of U  in 0:01:16.683421
encoded 1200 rows of M  in 0:01:19.523387
encoded 1200 rows of 8  in 0:01:17.303207
encoded 1200 rows of X  in 0:01:17.066256
encoded 1200 rows of K  in 0:01:18.035543
encoded 1200 rows of Q  in 0:01:17.572959
encoded 1200 rows of Y  in 0:01:18.207828
encoded 1200 rows of S  in 0:01:18.832753
encoded 1200 rows of G  in 0:01:19.088613
encoded 1200 rows of A  in 0:01:19.308903
encoded 1200 rows of O  in 0:01:18.579271
encoded 1200 rows of T  in 0:01:17.593544
encoded 1200 rows of V  in 0:01:20.316417
encoded 1200 rows of Z  in 0:01:21

In [3]:
import arrow
from umap import UMAP

time_start = arrow.now()
umap = UMAP(random_state=2024, verbose=True, n_jobs=1, low_memory=False, n_epochs=201)
df[['x', 'y']] = umap.fit_transform(X=df['value'].apply(func=pd.Series))
print('done with UMAP in {}'.format(arrow.now() - time_start))

2024-08-14 17:48:50.736738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 17:48:50.736965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 17:48:50.917589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(low_memory=False, n_epochs=201, n_jobs=1, random_state=2024, verbose=True)
Wed Aug 14 17:49:09 2024 Construct fuzzy simplicial set
Wed Aug 14 17:49:09 2024 Finding Nearest Neighbors
Wed Aug 14 17:49:09 2024 Building RP forest with 15 trees
Wed Aug 14 17:49:17 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	Stopping threshold met -- exiting after 3 iterations
Wed Aug 14 17:49:41 2024 Finished Nearest Neighbor Search
Wed Aug 14 17:49:47 2024 Construct embedding


Epochs completed:   0%|            0/201 [00:00]

	completed  0  /  201 epochs
	completed  20  /  201 epochs
	completed  40  /  201 epochs
	completed  60  /  201 epochs
	completed  80  /  201 epochs
	completed  100  /  201 epochs
	completed  120  /  201 epochs
	completed  140  /  201 epochs
	completed  160  /  201 epochs
	completed  180  /  201 epochs
	completed  200  /  201 epochs
Wed Aug 14 17:50:25 2024 Finished embedding
done with UMAP in 0:01:22.174556


In [4]:
from plotly import express

import warnings

warnings.filterwarnings(action='ignore', category=FutureWarning)

express.scatter(data_frame=df, x='x', y='y', color='tag', height=800)

This is really encourating; UMAP clusters our embeddings in such a way that almost(?) all the time a point's nearest neighbor is a picture of a sign for the same letter or number. Let's build a model. 

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(df['value'].apply(pd.Series), df['tag'], test_size=0.2, random_state=2024, stratify=df['tag'])
model = LogisticRegression(max_iter=1000, tol=1e-12).fit(X_train, y_train)
print('model fit in {} iterations'.format(model.n_iter_[0]))

print('accuracy: {:5.4f}'.format(accuracy_score(y_true=y_test, y_pred=model.predict(X=X_test))))

model fit in 469 iterations
accuracy: 1.0000


If our model has an accuracy of 1.0 there's no need to look at the classification report.